In [152]:
from datasets import load_dataset, Dataset
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

In [145]:
dataset = load_dataset('wikipedia', "20220301.en", split='train', streaming=True)
shuffled_dataset = dataset.shuffle(seed=42, buffer_size=10_000)

In [146]:
sel = dataset.take(50000)

In [149]:
selwikidata = list(sel)

In [8]:
TOTDATA = 60000
quotesdata = quotesdata['train'].shuffle(seed=0)
quotesdata = quotesdata.select(range(TOTDATA))

In [11]:
quotesdata[1]

{'quote': "The rose, however, made us girls somewhat fainthearted, because it really was something we felt mattered, the white bridal dream with the wedding bouquet and the kiss from the man who was to be ours forever. But then Laura said that the lady who had given it to us had gotten divorced only five years later. And since many of our parents were also divorce, if indeed they had ever been married at all, that dream clearly wasn't worth our time.",
 'author': 'Janne Teller',
 'category': 'marriage, nothing'}

In [134]:
modelname = "EleutherAI/gpt-neo-1.3B"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(modelname, device_map=0)
toker = AutoTokenizer.from_pretrained(modelname, padding_side='left')

toker.max_length=512
toker.padding_size='left'
toker.pad_token = toker.eos_token

In [139]:
def proc_quotes(exs):
    inps = []
    for e in exs:
        inps.append(e['author']+ ": " + e['quote'])
    return inps
        
def generate_trunc(inputs, trunc, model):
    newinps = []
    corrgens = []
    for inp in inputs: 
        newinps.append(toker.decode(toker(inp).input_ids[:-(trunc+1)], skip_special_tokens=True))
        corrgens.append(toker.decode(toker(inp).input_ids[-(trunc+1):], skip_special_tokens=True))
    inps = toker(newinps, padding=True, truncation=True, return_tensors="pt").to(model.device)
    newgens = model.generate(**inps, max_new_tokens=trunc+1, do_sample=True, top_p=0.8)
    return toker.batch_decode(newgens, skip_special_tokens=True), corrgens

In [140]:
procd = proc_quotes(quotesdata.select(range(10)))
gtrunc, golds = generate_trunc(procd, 3, model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [144]:
ind = 2
print("ORIGINAL: "+procd[ind])
print('GOLD: '+golds[ind])
print("GEN: "+gtrunc[ind])

ORIGINAL: Baris Gencel: At this stage I can see most of the men are still primitive about sexual perception and conception.
GOLD:  perception and conception.
GEN: Baris Gencel: At this stage I can see most of the men are still primitive about sexual desire and so there
